### IndPenSim Data websites 
Addition details can be found at:<a href='http://www.industrialpenicillinsimulation.com/'> www.industrialpenicillinsimulation.com</a>
Data is downloaded from : <a href='https://data.mendeley.com/datasets/pdnjz7zz5x/1'> Mendeley data Website</a>

In [2]:
## Import necessary packages
from urllib.request import urlretrieve 
import numpy as np 
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd

## Unzip IndPenSim Data

In [3]:
data = pd.read_csv('data/clean_data.csv')
print(data.columns)
data

Index(['Unnamed: 0', 'Time (h)', 'Aeration rate(Fg:L/h)',
       'Agitator RPM(RPM:RPM)', 'Sugar feed rate(Fs:L/h)',
       'Acid flow rate(Fa:L/h)', 'Base flow rate(Fb:L/h)',
       'Heating/cooling water flow rate(Fc:L/h)',
       'Heating water flow rate(Fh:L/h)',
       'Water for injection/dilution(Fw:L/h)',
       'Air head pressure(pressure:bar)', 'Dumped broth flow(Fremoved:L/h)',
       'Substrate concentration(S:g/L)',
       'Dissolved oxygen concentration(DO2:mg/L)',
       'Penicillin concentration(P:g/L)', 'Vessel Volume(V:L)',
       'Vessel Weight(Wt:Kg)', 'pH(pH:pH)', 'Temperature(T:K)',
       'Generated heat(Q:kJ)',
       'carbon dioxide percent in off-gas(CO2outgas:%)',
       'PAA flow(Fpaa:PAA flow (L/h))',
       'PAA concentration offline(PAA_offline:PAA (g L^{-1}))',
       'Oil flow(Foil:L/hr)',
       'NH_3 concentration off-line(NH3_offline:NH3 (g L^{-1}))',
       'Oxygen Uptake Rate(OUR:(g min^{-1}))',
       'Oxygen in percent in off-gas(O2:O2  (%))',
  

,Unnamed: 0,Time (h),Aeration rate(Fg:L/h),Agitator RPM(RPM:RPM),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),...,Ammonia shots(NH3_shots:kgs),Viscosity(Viscosity_offline:centPoise),Fault reference(Fault_ref:Fault ref),0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref),1- No Raman spec,1-Raman spec recorded,Batch reference(Batch_ref:Batch ref),2-PAT control(PAT_ref:PAT ref),Batch ID,Fault flag
0,0,0.2,30.0,100.0,8.0,0.0000,30.1180,9.8335,0.00010,0.0,...,0.0,NaN,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1,0.4,30.0,100.0,8.0,0.0000,51.2210,18.1550,0.00010,0.0,...,0.0,NaN,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2,2,0.6,30.0,100.0,8.0,0.0000,54.3020,9.5982,0.00010,0.0,...,0.0,NaN,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,3,0.8,30.0,100.0,8.0,0.0000,37.8160,4.3395,0.00010,0.0,...,0.0,NaN,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
4,4,1.0,30.0,100.0,8.0,0.5181,18.9080,1.1045,0.00010,0.0,...,0.0,4.083,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113930,113930,229.2,65.0,100.0,80.0,0.0000,12.9000,33.5860,5.93590,250.0,...,0.0,NaN,0.0,0.0,1.0,100.0,100.0,1.0,43854.0,44034.0
113931,113931,229.4,65.0,100.0,80.0,0.0000,10.0470,10.0760,98.83400,250.0,...,0.0,NaN,0.0,0.0,1.0,100.0,100.0,1.0,44814.0,44659.0
113932,113932,229.6,65.0,100.0,80.0,0.0000,7.3609,69.3450,9.88340,250.0,...,0.0,NaN,0.0,0.0,1.0,100.0,100.0,1.0,44273.0,43952.0
113933,113933,229.8,65.0,100.0,80.0,0.0000,6.2794,38.6100,0.98834,250.0,...,0.0,NaN,0.0,0.0,1.0,100.0,100.0,1.0,44733.0,45245.0



## Data Processing